<a href="https://colab.research.google.com/github/jabo16ac/Empirical-analysis-of-investors-perceptions-of-CoCos/blob/main/Empirical_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Empirical analysis of investors’ perceptions of CoCos: Are they going concern capital?

## 1 Imports
Libraries are imported

In [87]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats

## 2 Processing
Data is prepared for statistical analysis

### 2.1 Financial data
The financial data is imported and processed

In [88]:
# Import financial data data
df = pd.read_csv('03_Financials.csv', header=0)

# Set unique identifier (ISIN+Year) as index
df = df.set_index('UID')

# Exclude banks with assets less than EUR 1bn 
df = df[df['TOTAL ASSETS'] >= 10**9]

# Rename columns
df.columns = ['ISIN', 'Year', 'DERIVATIVES (ASSETS)', 'OTHER SECURITIES',
       'TOTAL ASSETS', 'DERIVATIVES (LIABILITIES AND EQUITY)',
       'OFF BALANCE SHEET ITEMS', '10_TotalCapitalRatio', 'TOTAL CAPITAL',
       'TIER 1 CAPITAL', '9_CostIncomeRatio',
       '2_NPLRatio', '8_ROA',
       '3_LoansRatio', 'Country']

# Re-format and calculate columns
df['RWA'] = df['TOTAL CAPITAL']/(df['10_TotalCapitalRatio']/100)
df['1_RWARatio'] = df['RWA']/df['TOTAL ASSETS']
df['Derivs'] = (df['DERIVATIVES (ASSETS)'] + df['DERIVATIVES (LIABILITIES AND EQUITY)'])
df['4_SecuritiesRatio'] = df['OTHER SECURITIES']/df['TOTAL ASSETS']
df['5_DerivativesRatio'] = df['Derivs']/df['TOTAL ASSETS']
df['2_NPLRatio'] = df['2_NPLRatio']/100
df['3_LoansRatio'] = df['3_LoansRatio']/100
df['6_OBSRatio'] = df['OFF BALANCE SHEET ITEMS']/df['TOTAL ASSETS']
df['7_Size'] = np.log(df['TOTAL ASSETS'])
df['8_ROA'] = df['8_ROA']/100
df['9_CostIncomeRatio'] = df['9_CostIncomeRatio']/100
df['10_TotalCapitalRatio'] = df['10_TotalCapitalRatio']/100
df['11_CapitalQuality'] = df['TIER 1 CAPITAL']/df['TOTAL CAPITAL']

# Drop old columns
df = df.drop(['DERIVATIVES (ASSETS)', 'OTHER SECURITIES', 'TOTAL ASSETS', 
         'DERIVATIVES (LIABILITIES AND EQUITY)', 'OFF BALANCE SHEET ITEMS', 
         'TOTAL CAPITAL', 'TIER 1 CAPITAL', 'RWA', 'Derivs',], axis = 1)

# Select these columns
columns = ['ISIN', 'Year', 'Country', '1_RWARatio', '2_NPLRatio', '3_LoansRatio', 
           '4_SecuritiesRatio', '5_DerivativesRatio', '6_OBSRatio', '7_Size', 
           '8_ROA', '9_CostIncomeRatio', '10_TotalCapitalRatio', '11_CapitalQuality']
financials = df[columns]

### 2.2 Risk data
The stock price data is imported and volatility, VaR, and ES is calculated.

In [89]:
### Import price and stock data
pdf = pd.read_csv('01_Stock price.csv', header=3) 
ddf = pd.read_csv('02_Dates.csv', header=3)

### Extract bank ISINs
banks = pdf.columns.values # Deduct bank names

### Calculate daily returns
rdf = (pdf - pdf.shift(1)) / pdf.shift(1) # Calculate returns

### Define variables
voldf = pd.DataFrame()
years = pd.Series(np.arange(2011, 2021))

### Run loop for every bank
for j, element in enumerate(banks):
  name = element 
  names = pd.Series(name, index = np.arange(0,10)) # List names x years
  
  # Clear variables 
  vols = [] 
  VaRs = []
  ESs = []

  ### Run loop for every every (by every bank)
  for i in range(2011, 2021):
    # Rename columns and join price and date
    y = rdf[banks[j]].rename('Return') 
    x = ddf[banks[j]].rename('Date')
    df = pd.concat([x,y], axis = 1).set_index(pd.to_datetime(x, dayfirst=True)).drop('Date', axis = 1)

    # Select relevant year
    year = i
    start = str(str(year) + "-01-01")
    end = str(str(year) + "-12-31")
    df = df[start:end]

    # Calculate daily volatility and mean
    daily_vol = df.std(ddof = 1)
    mean = np.mean(df)

    # Calculate annualsed volatility, VaR and ES
    vol = np.sqrt(len(df)) * daily_vol # Annualize given trading days where len(df) is the no. of trading days
    VaR = mean + daily_vol*scipy.stats.norm.ppf(0.025)
    ES = mean - daily_vol*(scipy.stats.norm.pdf(scipy.stats.norm.ppf(0.025)))/0.025
    
    # Append the yearly vol/VaR/ES to table for each bank
    vols = pd.Series(np.append(vols, [vol]))
    VaRs = pd.Series(np.append(VaRs, [VaR]))
    ESs  = pd.Series(np.append(ESs, [ES]))

  # Create dataframe with unique identifier
  d1 = pd.concat([names, years], axis = 1)

  # Set index to unique idenfier by joining columns
  d2 = d1[[0,1]].astype(str).apply(lambda x: '-'.join(x), axis=1) # Create index (bank + year)
  
  # Join all three columns (UID, ISIN, Year)
  df = pd.concat([d2, d1], axis = 1) 
  
  # Join above with calculated volatitilies, VaRs, and ES
  volatilities = pd.concat([df, vols, VaRs, ESs], axis = 1) 
  
  # Append to all previous volatility tables for each bank
  voldf = voldf.append(volatilities) 

# Rename columns and set UID as index
voldf.columns =  ['UID', 'ISIN', 'Year', '12_Volatility', '13_VaR', '14_ES']
voldf = voldf.set_index('UID')

### 2.3 Financial data
The financial data is imported and processed

In [90]:
### Load CoCo issuance data
ccc = pd.read_csv('04_CoCos.csv', header=0).set_index('UID')

### Merge financials, risk-metrics, and CoCo data
df = pd.merge(financials, voldf.iloc[:,2:], left_index=True, right_index=True)
df = pd.merge(df, ccc, left_index=True, right_index=True)

### 2.4 Data manipulation
Control variables are lagged

In [91]:
### Pre-define dataframe
shiftedcontrols = pd.DataFrame()

### Get unique bank names
banks = df['ISIN'].unique()

### Declare columns to shift
shiftcols = ['7_Size',
            '3_LoansRatio',
            '8_ROA',
            '9_CostIncomeRatio', 
            '10_TotalCapitalRatio', 
            '11_CapitalQuality']

### Lag control variables
for j, element in enumerate(banks):
  bankdf = df[df['ISIN']==element]
  bankdf[shiftcols] = bankdf[shiftcols].shift(1)
  shiftedcontrols = shiftedcontrols.append(bankdf).sort_index()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


### 2.5 Outlier exclusion
Control variables are lagged

In [92]:
### Declare columns to drop NA's in, then drop them
dropcols   = ['1_RWARatio',
              '3_LoansRatio',
              '7_Size',
              '8_ROA',
              '9_CostIncomeRatio', 
              '10_TotalCapitalRatio', 
              '11_CapitalQuality',
              '12_Volatility',
              '13_VaR',
              '14_ES']
df = shiftedcontrols.dropna(subset=dropcols)

### Manual outlier exclusion
df = df[df['1_RWARatio'] < 1]
df = df[df['2_NPLRatio'] < 1]
df = df[df['3_LoansRatio'] < 1]
df = df[df['4_SecuritiesRatio'] < 1]
df = df[df['5_DerivativesRatio'] < 1]
df = df[df['6_OBSRatio'] < 1]
df = df[df['8_ROA'] < 0.5]
df = df[df['9_CostIncomeRatio'] < 1.5]
df = df[df['10_TotalCapitalRatio'] < .5]
df = df[df['11_CapitalQuality'] < 1.5]
df = df[df['12_Volatility'] < 2]

### Exclude all banks which do not have atleast five years of data
selection = df['ISIN'].value_counts()>4
df = df[df['ISIN'].isin(selection.index[selection].values)]

### 2.6 Descriptive statistics

Net sample is tabulated across country and year

In [93]:
bank_year_obs = pd.pivot_table(df, values='ISIN', index=['Country'], columns=['Year'], aggfunc=len, fill_value=0)
#print(bank_year_obs.to_latex(index=True)) # Used to export to Latex
bank_year_obs

Year,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0
Country,,,,,,,,
AT,5,5,6,6,6,5,6,6
BE,2,1,2,2,2,2,2,2
CH,9,11,14,14,17,16,17,16
CY,1,1,1,1,1,1,1,1
CZ,1,1,1,1,1,1,1,1
DE,4,4,5,6,6,5,6,6
DK,8,8,9,10,10,10,10,10
ES,6,6,6,6,7,6,7,7
FI,2,2,2,2,2,2,2,2


Averages and total for above table are calculated

In [94]:
print("Total obs. per country:\n", bank_year_obs.sum(axis=1))
print("Average obs. per country:", bank_year_obs.sum(axis=1).sum())

print("\nTotal obs. per year:\n", bank_year_obs.sum(axis=0))
print("Average obs. per year:", bank_year_obs.sum(axis=1).mean())

print("\nTotal obs:", bank_year_obs.sum().sum())

Total obs. per country:
 Country
AT     45
BE     15
CH    114
CY      8
CZ      8
DE     42
DK     75
ES     51
FI     16
FR     28
GB     66
GR     21
HR      8
HU      8
IE     13
IT    150
LI      8
LT      5
MT     13
NL     17
NO    122
PL     53
PT      7
RO      6
SE     24
SK     28
dtype: int64
Average obs. per country: 951

Total obs. per year:
 Year
2012.0     99
2013.0    109
2014.0    116
2015.0    124
2016.0    128
2017.0    121
2018.0    128
2019.0    126
dtype: int64
Average obs. per year: 36.57692307692308

Total obs: 951


CoCo sample is tabulated across country and year

In [95]:
### Dataframe is pre-defined
CoCos = pd.DataFrame()

### Year and country is selected from main data frame
CoCos["Country"] = df.index.str.slice(0,2)
CoCos["Year"] = df.index.str.slice(13,13+4).astype('int32')

### CoCo indicator variables are extracted from main data frame
CoCos["CoCo_EC"] = df['CoCo_EC'].values
CoCos["CoCo_PW"] = df['CoCo_PW'].values
CoCos["CoCo"] = df['CoCo'].values

### EC CoCos
#CoCo_obs = pd.pivot_table(CoCos, values='CoCo_EC', index=['Country'], columns=['Year'], aggfunc=sum, fill_value=0)
#print(CoCo_obs.to_latex(index=True))

### WD CoCos
#CoCo_obs = pd.pivot_table(CoCos, values='CoCo_PW', index=['Country'], columns=['Year'], aggfunc=sum, fill_value=0)
#print(CoCo_obs.to_latex(index=True))

### All CoCos
CoCo_obs = pd.pivot_table(CoCos, values='CoCo', index=['Country'], columns=['Year'], aggfunc=sum, fill_value=0)
#print(CoCo_obs.to_latex(index=True)) # Used to export to Latex
CoCo_obs

Year,2012,2013,2014,2015,2016,2017,2018,2019
Country,,,,,,,,
AT,0,0,1,2,1,2,2,3
BE,0,0,1,0,0,0,1,1
CH,1,0,1,3,2,2,1,3
CY,0,1,0,0,0,0,0,0
CZ,0,0,0,0,0,0,0,0
DE,0,0,1,0,0,0,1,1
DK,0,0,0,1,2,1,1,1
ES,0,1,1,1,2,3,3,1
FI,0,0,0,0,0,0,0,0


Descriptive statistics for all continous variables are calculated

In [96]:
#print(df.describe().transpose().to_latex(float_format="%.4f")) # Used to export to Latex
df.describe()

,Year,1_RWARatio,2_NPLRatio,3_LoansRatio,4_SecuritiesRatio,5_DerivativesRatio,6_OBSRatio,7_Size,8_ROA,9_CostIncomeRatio,10_TotalCapitalRatio,11_CapitalQuality,12_Volatility,13_VaR,14_ES,Pre-CoCo,CoCo,Post-CoCo,Pre-CoCo_EC,CoCo_EC,Post-CoCo_EC,Pre-CoCo_PW,CoCo_PW,Post-CoCoPW
count,951.000000,951.000000,951.000000,951.000000,951.000000,951.000000,951.000000,951.000000,951.000000,951.000000,951.000000,951.000000,951.000000,951.000000,951.000000,951.000000,951.000000,951.000000,951.000000,951.000000,951.000000,951.000000,951.000000,951.000000
mean,2015.659306,0.473624,0.071217,0.590234,0.204611,0.060896,0.143553,24.319865,0.004953,0.597627,0.168970,0.887682,0.308767,-0.037962,-0.045325,0.141956,0.126183,0.098843,0.062040,0.055731,0.046267,0.126183,0.111462,0.086225
std,2.250056,0.168228,0.096838,0.197161,0.139298,0.105053,0.132458,1.974048,0.011661,0.337895,0.041424,0.094557,0.205328,0.025857,0.030786,0.349188,0.332230,0.298609,0.241355,0.229522,0.210173,0.332230,0.314869,0.280844
min,2012.000000,0.091005,0.000600,0.032500,0.002224,0.000000,0.000023,20.737268,-0.134100,-5.253300,-0.050000,0.583944,0.000000,-0.235977,-0.277759,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2014.000000,0.344624,0.013450,0.480600,0.117191,0.006157,0.044284,23.000299,0.002300,0.508800,0.142600,0.828764,0.203226,-0.042747,-0.050929,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2016.000000,0.475121,0.035100,0.633600,0.178273,0.019276,0.119893,23.967856,0.005300,0.601400,0.163400,0.894243,0.262336,-0.032147,-0.038267,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2018.000000,0.596513,0.082350,0.737450,0.249614,0.066344,0.198596,25.682602,0.008900,0.711700,0.187150,0.967655,0.347445,-0.024570,-0.029450,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2019.000000,0.983717,0.615600,0.902500,0.829269,0.750187,0.997831,28.660641,0.179900,1.284200,0.343000,1.357402,1.937942,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## 3 Analysis

### 3.1 Asset risk-shifting test

#### 3.1.1 All CoCos
Regressions are run for the six asset risk-shifiting metrics against all CoCos

In [97]:
res1 = smf.ols(formula='Q("1_RWARatio") ~ Q("Post-CoCo") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res1.summary())

res2 = smf.ols(formula='Q("2_NPLRatio") ~ Q("Post-CoCo") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res2.summary())

res3 = smf.ols(formula='Q("3_LoansRatio") ~ Q("Post-CoCo") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res3.summary())

res4 = smf.ols(formula='Q("4_SecuritiesRatio") ~ Q("Post-CoCo") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res4.summary())

res5 = smf.ols(formula='Q("5_DerivativesRatio") ~ Q("Post-CoCo") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res5.summary())

res6 = smf.ols(formula='Q("6_OBSRatio") ~ Q("Post-CoCo") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res6.summary())

                            OLS Regression Results                            
Dep. Variable:        Q("1_RWARatio")   R-squared:                       0.904
Model:                            OLS   Adj. R-squared:                  0.887
Method:                 Least Squares   F-statistic:                     53.21
Date:                Fri, 12 Mar 2021   Prob (F-statistic):               0.00
Time:                        00:58:36   Log-Likelihood:                 1461.0
No. Observations:                 951   AIC:                            -2634.
Df Residuals:                     807   BIC:                            -1935.
Df Model:                         143                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

#### 3.1.2 EC CoCos
Regressions are run for the six asset risk-shifiting metrics against EC CoCos

In [98]:
res1 = smf.ols(formula='Q("1_RWARatio") ~ Q("Post-CoCo_EC") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res1.summary())

res2 = smf.ols(formula='Q("2_NPLRatio") ~ Q("Post-CoCo_EC") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res2.summary())

res3 = smf.ols(formula='Q("3_LoansRatio") ~ Q("Post-CoCo_EC") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res3.summary())

res4 = smf.ols(formula='Q("4_SecuritiesRatio") ~ Q("Post-CoCo_EC") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res4.summary())

res5 = smf.ols(formula='Q("5_DerivativesRatio") ~ Q("Post-CoCo_EC") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res5.summary())

res6 = smf.ols(formula='Q("6_OBSRatio") ~ Q("Post-CoCo_EC") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res6.summary())

                            OLS Regression Results                            
Dep. Variable:        Q("1_RWARatio")   R-squared:                       0.904
Model:                            OLS   Adj. R-squared:                  0.887
Method:                 Least Squares   F-statistic:                     53.16
Date:                Fri, 12 Mar 2021   Prob (F-statistic):               0.00
Time:                        00:58:37   Log-Likelihood:                 1460.6
No. Observations:                 951   AIC:                            -2633.
Df Residuals:                     807   BIC:                            -1934.
Df Model:                         143                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

#### 3.1.3 PW CoCos

Regressions are run for the six asset risk-shifiting metrics against PW CoCos

In [99]:
res1 = smf.ols(formula='Q("1_RWARatio") ~ Q("Post-CoCoPW") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res1.summary())

res2 = smf.ols(formula='Q("2_NPLRatio") ~ Q("Post-CoCoPW") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res2.summary())

res3 = smf.ols(formula='Q("3_LoansRatio") ~ Q("Post-CoCoPW") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res3.summary())

res4 = smf.ols(formula='Q("4_SecuritiesRatio") ~ Q("Post-CoCoPW") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res4.summary())

res5 = smf.ols(formula='Q("5_DerivativesRatio") ~ Q("Post-CoCoPW") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res5.summary())

res6 = smf.ols(formula='Q("6_OBSRatio") ~ Q("Post-CoCoPW") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res6.summary())

                            OLS Regression Results                            
Dep. Variable:        Q("1_RWARatio")   R-squared:                       0.904
Model:                            OLS   Adj. R-squared:                  0.887
Method:                 Least Squares   F-statistic:                     53.17
Date:                Fri, 12 Mar 2021   Prob (F-statistic):               0.00
Time:                        00:58:38   Log-Likelihood:                 1460.7
No. Observations:                 951   AIC:                            -2633.
Df Residuals:                     807   BIC:                            -1934.
Df Model:                         143                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

### 3.2 Primary empirical model

#### 3.2.1. All CoCos
Regressions are run for the three risk-metrics against all CoCos

In [111]:
res1 = smf.ols(formula='Q("12_Volatility") ~ Q("Pre-CoCo") + Q("CoCo") + Q("Post-CoCo") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res1.summary())


#res4 = smf.ols(formula='Q("12_Volatility") ~ Q("Pre-CoCo_EC") + Q("Post-CoCo_EC") + Q("Pre-CoCo_PW") + Q("CoCo_PW") + Q("Post-CoCoPW") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + Q("3_LoansRatio") + ISIN + C(Year)', 
#               data=df).fit()
#print(res4.summary())



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     Q("12_Volatility")   R-squared:                       0.672
Model:                            OLS   Adj. R-squared:                  0.613
Method:                 Least Squares   F-statistic:                     11.38
Date:                Fri, 12 Mar 2021   Prob (F-statistic):          6.02e-122
Time:                        01:08:23   Log-Likelihood:                 686.96
No. Observations:                 951   AIC:                            -1082.
Df Residuals:                     805   BIC:                            -372.7
Df Model:                         145                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                    -1.7748      0.703     -2.525      0.012      -3.155      -0.395
ISIN[T.AT0000624705]         -0.0894      0.103     -0.870      0.385      -0.291       0.112
ISIN[T.AT0000625108]         -0.1956      0.085     -2.314      0.021      -0.362      -0.030
ISIN[T.AT0000625504]         -0.1137      0.097     -1.167      0.243      -0.305       0.078
ISIN[T.AT0000652011]         -0.1029      0.065     -1.577      0.115      -0.231       0.025
ISIN[T.AT0000824701]          0.2820      0.128      2.205      0.028       0.031       0.533
ISIN[T.BE0003565737]         -0.1541      0.067     -2.285      0.023      -0.287      -0.022
ISIN[T.BE0974290224]          0.2028      0.088      2.294      0.022       0.029       0.376
ISIN[T.CH0001308904]         -0.1126      0.097     -1.158      0.247      -0.303       0.078
ISIN[T.CH0001340204]         -0.1628      0.088     -1.855      0.064      -0.335       0.009
ISIN[T.CH0001473559]         -0.1840      0.087     -2.110      0.035      -0.355      -0.013
ISIN[T.CH0001642682]         -0.1340      0.090     -1.494      0.136      -0.310       0.042
ISIN[T.CH0009236461]         -0.1260      0.081     -1.561      0.119      -0.284       0.032
ISIN[T.CH0010675863]          0.1931      0.122      1.580      0.114      -0.047       0.433
ISIN[T.CH0011484067]         -0.1253      0.079     -1.590      0.112      -0.280       0.029
ISIN[T.CH0011693600]         -0.1796      0.082     -2.179      0.030      -0.341      -0.018
ISIN[T.CH0012138530]         -0.2626      0.081     -3.234      0.001      -0.422      -0.103
ISIN[T.CH0012335540]         -0.0276      0.088     -0.315      0.753      -0.199       0.144
ISIN[T.CH0014786500]         -0.0753      0.086     -0.872      0.384      -0.245       0.094
ISIN[T.CH0015251710]         -0.1743      0.076     -2.296      0.022      -0.323      -0.025
ISIN[T.CH0022268228]          0.1005      0.082      1.220      0.223      -0.061       0.262
ISIN[T.CH0102484968]         -0.1082      0.073     -1.482      0.139      -0.251       0.035
ISIN[T.CH0189396655]          0.0392      0.117      0.335      0.738      -0.190       0.268
ISIN[T.CH0231351104]         -0.1290      0.090     -1.427      0.154      -0.306       0.048
ISIN[T.CH0305951201]         -0.0946      0.097     -0.979      0.328      -0.284       0.095
ISIN[T.CY0105570119]          0.2219      0.100      2.214      0.027       0.025       0.419
ISIN[T.CZ0008019106]         -0.0977      0.077     -1.270      0.205      -0.249       0.053
ISIN[T.DE0005140008]         -0.2663      0.095     -2.810      0.005      -0.452      -0.080
ISIN[T.DE0005408116]         -0.0124      0.074     -0.167      0.868      -0.158       0.133
ISIN[T.DE0008019001]         -0.0807      0.077     -1.044      0.297      

#### 3.2.2. WD CoCos

Regressions are run for the three risk-metrics against WD CoCos

In [100]:
res2 = smf.ols(formula='Q("12_Volatility") ~ Q("Pre-CoCo_PW") + Q("CoCo_PW") + Q("Post-CoCoPW") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res2.summary())

#### 3.2.3. EC CoCos

Regressions are run for the three risk-metrics against EC CoCos

In [100]:
res3 = smf.ols(formula='Q("12_Volatility") ~ Q("Pre-CoCo_EC") + Q("CoCo_EC") + Q("Post-CoCo_EC") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res3.summary())